Proprietary content. ©Great Learning. All Rights Reserved. Unauthorized use or distribution prohibited

### Libraries along with their versions used at the time of making notebook-
google	2.0.3

numpy	1.18.1

pandas	0.25.3

tensorflow	2.1.0

Firstly, let's select TensorFlow version 2.x in colab

In [1]:
%tensorflow_version 2.x
import tensorflow
tensorflow.__version__

'2.8.0'

In [2]:
# Initialize the random number generator
import random
random.seed(0)

# Ignore the warnings
import warnings
warnings.filterwarnings("ignore")

### Load the dataset

As we are using google colab, we need to mount the google drive to load the data file

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
!ls /content/drive/MyDrive/MTech/LSTM_Code_data/


 ner_dataset-1.csv		      Sentiment-1.csv
 POS_tagging_using_LSTM.ipynb	      Sentiment_Analysis_using_LSTM.ipynb
'S4_Faculty_Notebook_nlp_(1).ipynb'


set project path

In [5]:
project_path = '/content/drive/MyDrive/MTech/LSTM_Code_data/'

In [6]:
import pandas as pd

data = pd.read_csv( project_path + 'Sentiment-1.csv')
# Keeping only the neccessary columns
data = data[['text','sentiment']]

In [7]:
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

data = data[data.sentiment != "Neutral"]
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(data[ data['sentiment'] == 'Positive'].size)
print(data[ data['sentiment'] == 'Negative'].size)

for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')
    
vocabSize = 2000
tokenizer = Tokenizer(num_words=vocabSize, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

4472
16986


In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D

embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(vocabSize, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 28, 128)           256000    
                                                                 
 spatial_dropout1d (SpatialD  (None, 28, 128)          0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 196)               254800    
                                                                 
 dense (Dense)               (None, 2)                 394       
                                                                 
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [9]:
from sklearn.model_selection import train_test_split

Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.15, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(9119, 28) (9119, 2)
(1610, 28) (1610, 2)


In [10]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 10, batch_size=batch_size, verbose = 2)

Epoch 1/10
285/285 - 37s - loss: 0.4209 - accuracy: 0.8260 - 37s/epoch - 129ms/step
Epoch 2/10
285/285 - 32s - loss: 0.3121 - accuracy: 0.8685 - 32s/epoch - 112ms/step
Epoch 3/10
285/285 - 32s - loss: 0.2734 - accuracy: 0.8873 - 32s/epoch - 114ms/step
Epoch 4/10
285/285 - 32s - loss: 0.2446 - accuracy: 0.8979 - 32s/epoch - 111ms/step
Epoch 5/10
285/285 - 32s - loss: 0.2233 - accuracy: 0.9074 - 32s/epoch - 111ms/step
Epoch 6/10
285/285 - 32s - loss: 0.2038 - accuracy: 0.9174 - 32s/epoch - 112ms/step
Epoch 7/10
285/285 - 32s - loss: 0.1909 - accuracy: 0.9228 - 32s/epoch - 111ms/step
Epoch 8/10
285/285 - 32s - loss: 0.1722 - accuracy: 0.9295 - 32s/epoch - 111ms/step
Epoch 9/10
285/285 - 32s - loss: 0.1571 - accuracy: 0.9349 - 32s/epoch - 112ms/step
Epoch 10/10
285/285 - 32s - loss: 0.1497 - accuracy: 0.9384 - 32s/epoch - 112ms/step


In [11]:
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

51/51 - 1s - loss: 0.4528 - accuracy: 0.8348 - 913ms/epoch - 18ms/step
score: 0.45
acc: 0.83


In [12]:
import numpy as np

pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0

for x in range(len(X_test)):
    
    result = model.predict(X_test[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 1)[0]
   
    if np.argmax(result) == np.argmax(Y_test[x]):
        if np.argmax(Y_test[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_test[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1

print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

1/1 [==============================] - 0s 20ms/step
pos_acc 52.45398773006135 %
neg_acc 91.35514018691589 %


In [13]:
twt = ['He is a lazy person.']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0  32   5   7 374]]
1/1 - 0s - 20ms/epoch - 20ms/step
negative
